In [54]:
import pandas as pd
from collections import Counter
import re
import numpy as np

In [55]:
df_appen = pd.read_excel('appen/Query Classification_12.30.22.xlsx').sort_values('label_ordering')

In [56]:
df_original = pd.read_json("../query/top_used_wish_queries_offshore_sample_100000.json", lines=True)

In [57]:
files = [ 
    'offshore/TieBreaker File - 8 Dec 22.xlsx',
    'offshore/TieBreaker File - 9 Dec 22.xlsx',
    'offshore/TieBreaker File - 12 Dec 22.xlsx',
    'offshore/TieBreaker File - 13 Dec 22.xlsx',
    'offshore/TieBreaker File - 14 Dec 22.xlsx',
    'offshore/TieBreaker File - 15 Dec 22.xlsx',
    'offshore/TieBreaker File - 16 Dec 22.xlsx',
    'offshore/TieBreaker File - 19 Dec 22.xlsx',
    'offshore/TieBreaker File - 20 Dec 22.xlsx',
    'offshore/TieBreaker File - 21 Dec 22.xlsx',
    'offshore/TieBreaker File - 22 Dec 22.xlsx',
    'offshore/TieBreaker File - 23 Dec 22.xlsx',
    'offshore/TieBreaker File - 26 Dec 22.xlsx',
    'offshore/TieBreaker File - 27 _ 28 Dec 22.xlsx',
    'offshore/TieBreaker File - 29 Dec.xlsx',
    'offshore/TieBreaker File - 30 Dec 22 to 1 Jan 23.xlsx'
]
dfs = []
for i in files:
    tmp = pd.read_excel(i)
    tmp['filename'] = i
    dfs.append(tmp)

df_offshore = pd.concat(dfs)[['index', 'filename', 'Sr No', 'query', 'sample_method', 'gmv', 'cnt', 
    'Tiebreaked All SortedByConfidenceHighestFirst taxonomy path Full Paths']].rename( 
        columns={
            'Sr No': 'label_ordering', 
            'Tiebreaked All SortedByConfidenceHighestFirst taxonomy path Full Paths': 'query_classifications'}
    )


df_offshore['label_ordering'] = df_offshore['label_ordering'].astype(int)
df_offshore['index'] = df_offshore['index'].astype(int)
df_offshore['query'] = df_offshore['query'].astype(str)
df_offshore = df_offshore.merge(df_original[['index', 'query']].rename(columns={'query': 'orig_query'}), on='index', how='inner')
df_offshore.loc[df_offshore['query'] != df_offshore['orig_query'], "query"] = df_offshore.loc[df_offshore['query'] != df_offshore['orig_query'], "orig_query"]
assert len(df_offshore) == len(set(df_offshore['query']))


In [58]:
df_tax = pd.read_json('../taxonomy/wish_newtax.json', lines=True)

In [59]:
df_tax = df_tax[df_tax.category_path.apply(len) > 0]

In [60]:
df_tax = pd.read_json('../taxonomy/wish_newtax.json', lines=True)
df_tax = df_tax[df_tax.category_path.apply(len) > 0]
errors = []
class Trie:
    def __init__(self):
        self.trie = {}

    def add_text(self, text):
        node = self.trie
        for ind, i in enumerate(text):
            if i not in node:
                node[i] = {}
            node = node[i]
            if ind == len(text) - 1:
                node['<EOS>'] = 1
    
    def extract_from_text(self, text):
        node = self.trie
        res = []
        cur_res = ''
        for ind, i in enumerate(text):
            if i in node:
                node = node[i]
                cur_res += i
            else:
                try:
                    assert '<EOS>' in node
                except:
                    print('ERROR: ', text, res)
                    errors.append(text)
                    return None
                res.append(cur_res)
                cur_res = ''
                node = self.trie
        if len(cur_res) > 0:
            res.append(cur_res)
        
        return res
trie = Trie()
for i in df_tax.category_path.apply(lambda x: x.strip()).tolist():
    trie.add_text(i)
for i in df_tax.category_path.apply(lambda x: x.strip().lower()).tolist():
    trie.add_text(i)
trie.add_text('nan')
trie.add_text('No Categories Match')
manual_correction = {
    "Beauty & Health > Sexual Wellness > Sex Toys,Beauty & Health > Sexual Wellness > SM Products ,nan": \
        "Beauty & Health > Sexual Wellness > Sex Toys,Beauty & Health > Sexual Wellness > SM Products,nan",
    "Apparel Accessories > Girls' Accessories > Girls' Hair Accessories -,Apparel Accessories > Women's Hair Accessories,sports > fitness & body building > yoga > yoga hair bands": \
        "Apparel Accessories > Girls' Accessories > Girls' Hair Accessories,Apparel Accessories > Women's Hair Accessories,sports > fitness & body building > yoga > yoga hair bands",
    "Women's Clothing > Dresses ,women's clothing > weddings & events > wedding dresses,mother & kids > pregnancy & maternity > dresses": \
        "Women's Clothing > Dresses,women's clothing > weddings & events > wedding dresses,mother & kids > pregnancy & maternity > dresses",
    "Women's Clothing > Dresses, Mother & Kids > Girls' Baby Clothing > Dresses,Mother & Kids > Pregnancy & Maternity > Dresses": \
        "Women's Clothing > Dresses,Mother & Kids > Girls' Baby Clothing > Dresses,Mother & Kids > Pregnancy & Maternity > Dresses", 
    "Home & Garden > Arts, Crafts & Sewing > Apparel Sewing & Fabric > Buttons,Retrieving data. Wait a few seconds and try to cut or copy again.,nan": \
        "Home & Garden > Arts, Crafts & Sewing > Apparel Sewing & Fabric > Buttons,nan,nan",
    "Men's Clothing > Tops & Tees > T-Shirts,Watches > Men's Watches > Quartz Watch,Home & Garden > Kitchen,Dining & Bar > Coffeeware > Coffee Cups & Mugs": \
         "Men's Clothing > Tops & Tees > T-Shirts,Watches > Men's Watches > Quartz Watches,Home & Garden > Kitchen,Dining & Bar > Coffeeware > Coffee Cups & Mugs",
    "education & office supplies > writing & correction supplies > stencils,Crafts & Sewing > Scrapbooking & Stamping > Cutting Dies,home & garden > arts, crafts & sewing > apparel sewing & fabric > diy craft supplies": \
        "education & office supplies > writing & correction supplies > stencils,Home & Garden > Arts, Crafts & Sewing > Scrapbooking & Stamping > Cutting Dies,home & garden > arts, crafts & sewing > apparel sewing & fabric > diy craft supplies",
    "Home & Garden > Home Decor > Painting & Calligraphy,Home Improvement > Painting Supplies & Wall Treatments > Wallpapers,Home & Garden > Home Decor > Wall Sticker": \
        "Home & Garden > Home Decor > Painting & Calligraphy,Home Improvement > Painting Supplies & Wall Treatments > Wallpapers,Home & Garden > Home Decor > Wall Stickers",
    "Computer & Office > Office Electronics > Printer,Computer & Office > Office Electronics > All in One Printer,Education & Office Supplies > Paper > Printer Paper": \
        "Computer & Office > Office Electronics > Printers,Computer & Office > Office Electronics > All in One Printer,Education & Office Supplies > Paper > Printer Paper",
    "Home & Garden > Home Textile > Bedding > Pillow Cases, Home & Garden > Home Textile > Bedding > Quilts,Home & Garden > Home Textile > Duvet Cover Sets": \
        "Home & Garden > Home Textile > Bedding > Pillow Cases,Home & Garden > Home Textile > Bedding > Quilts,Home & Garden > Home Textile > Duvet Cover Sets",
    "Accessories > Earrings >\xa0Hoop Earrings,nan,nan": "Jewelry & Accessories > Earrings > Hoop Earrings,nan,nan", 
    "Shoes > Men's Shoes > Men's Boots,Shoes > Women's Shoes > Women's Boots,Children's Shoes > Boys > Boots Mother & Kids > Children's Shoes > Boys > Boots": \
        "Shoes > Men's Shoes > Men's Boots,Shoes > Women's Shoes > Women's Boots,Mother & Kids > Children's Shoes > Boys > Boots",
    "Beauty & Health > Sexual Wellness > SM Products > Bondage Gear,Beauty & Health > Sexual Wellness ,nan": \
        "Beauty & Health > Sexual Wellness > SM Products > Bondage Gear,Beauty & Health > Sexual Wellness,nan",
    "No Category Match": "No Categories Match,nan,nan",
    "car assessories interior,Mother & Kids > Car Seats & Accessories > Head & Body Supports,Mother & Kids > Car Seats & Accessories > Rear Facing Mirrors": \
        "Mother & Kids > Car Seats & Accessories > Head & Body Supports,Mother & Kids > Car Seats & Accessories > Rear Facing Mirrors,nan",
    "No Category Match,nan,nan": "No Categories Match,nan,nan",
    "Education & Office Supplies > Books & Magazines ,nan,nan": "Education & Office Supplies > Books & Magazines,nan,nan",
    "Apparel Accessories > Men's Accessories > Men's Masks,Apparel Accessories > Women's Accessories > Women's Masks  ,nan": \
        "Apparel Accessories > Men's Accessories > Men's Masks,Apparel Accessories > Women's Accessories > Women's Masks,nan",
    "Women's Clothing > Dresses ,Women's Clothing > Sweaters > Dresses,nan": \
        "Women's Clothing > Dresses,Women's Clothing > Sweaters > Dresses,nan",
    "Women's Clothing > Tops > Blouses & Shirts - ,nan,nan": "Women's Clothing > Tops > Blouses & Shirts,nan,nan",
    "Apparel Accessories > Girls' Accessories > Girls' Hair Accessories -,nan,nan": \
        "Apparel Accessories > Girls' Accessories > Girls' Hair Accessories,nan,nan",
    "Women's Clothing > Tops > Blouses & Shirts ,Women's Clothing > Bottoms > Pants,Women's Clothing > Dresses": \
        "Women's Clothing > Tops > Blouses & Shirts,Women's Clothing > Bottoms > Pants,Women's Clothing > Dresses",
    "Women's Clothing > Women's Exotic Apparel > Lingerie sets,nan,nan": \
        "Women's Clothing > Women's Exotic Apparel > Lingerie Sets,nan,nan",
    "Home & Garden > Garden Supplies > Pest Control > Foggers & Sprayers,\xa0,": \
        "Home & Garden > Garden Supplies > Pest Control > Foggers & Sprayers,nan,nan", 
    "Home & Garden > Kitchen,Dining & Bar > Kitchen Knives & Accessories > Blocks & Roll Bags, Home & Garden > Kitchen,Dining & Bar > Kitchen Knives & Accessories,nan": \
        "Home & Garden > Kitchen,Dining & Bar > Kitchen Knives & Accessories > Blocks & Roll Bags,Home & Garden > Kitchen,Dining & Bar > Kitchen Knives & Accessories,nan",
    "Automobiles & motorcycles > Auto replacement parts > exterior parts > chromium styling,Automobiles & Motorcycles > Auto Replacement Parts > Air Conditioning & Heat > Car Air Conditioning,nan": \
        "Automobiles & Motorcycles > Auto Replacement Parts > Exterior Parts > Chromium Styling,Automobiles & Motorcycles > Auto Replacement Parts > Air Conditioning & Heat > Car Air Conditioning,nan",
    "home improvement > bathroom fixtures > bathroom sinks, faucets & accessories,Home Decor > Bathroom >\xa0Faucets,nan": \
        "Home Improvement > Bathroom Fixtures > Bathroom Sinks, Faucets & Accessories > Basin Faucets,nan,nan",
    "Home Improvement > Bathroom Fixtures > Bathroom Sinks, Faucets & Accessories,Home Decor > Bathroom > Faucets,nan": \
        "Home Improvement > Bathroom Fixtures > Bathroom Sinks, Faucets & Accessories > Basin Faucets,nan,nan",
    "Women's Clothing > Dresses - ,Women's Clothing > Weddings & Events > Cocktail Dresses,Women's Clothing > Weddings & Events > Evening Dresses": \
        "Women's Clothing > Dresses,Women's Clothing > Weddings & Events > Cocktail Dresses,Women's Clothing > Weddings & Events > Evening Dresses",
    "Consumer Electronics > Smart Electronics > Wearable Devices > Smart watches,nan,nan": \
        "Consumer Electronics > Smart Electronics > Wearable Devices > Smart Watches,nan,nan",
    "Apparel Accessories > Men's Accessories > Men's Masks,Apparel Accessories > Women's Accessories > Women's Masks - ,Sports > Sports Accessories > Sports Face Masks > Cycling Face Mask": \
        "Apparel Accessories > Men's Accessories > Men's Masks,Apparel Accessories > Women's Accessories > Women's Masks,Sports > Sports Accessories > Sports Face Masks > Cycling Face Mask", 
    "No match category,nan,nan": "No Categories Match,nan,nan", 
    "Automotive > Parts & Accessories >\xa0Car Accessories,nan,nan": "Automobiles & Motorcycles > Interior Accessories,Automobiles & Motorcycles > Exterior Accessories,nan",
    "Sports > Fishing > Fishing Tools,sports > Fishing > Fishing Accessories,nan": \
        "Sports > Fishing > Fishing Tools,Sports > Fishing > Fishing Accessories,nan",
    "Hobbies > Outdoor & Camping Accessories >\xa0Sports Equipment,nan,nan": \
        "Sports,nan,nan",
    "Toys & hobbies > diecast & toy vehicles,toys & hobbies > play vehicles & models,toys & hobbies > play vehicles & models > railed, motor cars & bicycles": \
        "Toys & Hobbies > Diecast & Toy Vehicles,Toys & Hobbies > Play Vehicles & Models,Toys & Hobbies > Play Vehicles & Models > Railed, Motor Cars & Bicycles",
    "Hobbies > Outdoor & Camping Accessories > Sports Equipment,nan,nan": "Sports,nan,nan", 
    "Consumer electronics > power source > batteries,nan,nan": "Consumer Electronics > Power Source > Batteries,nan,nan",
    "Beauty & Health > Oral Hygiene > Teeth Whitening,Beauty & Health > oral hygiene > dental supplies > dental basic instrument,Beauty & Health > oral hygiene > dental supplies": \
        "Beauty & Health > Oral Hygiene > Teeth Whitening,Beauty & Health > Oral Hygiene > Dental Supplies > Dental Basic Instrument,Beauty & Health > Oral Hygiene > Dental Supplies",
    "Home & Garden > pet products > pet health care & hygiene > supplements & vitamins,Beauty & Health > Health Care > Personal Health Care,nan": \
        "Home & Garden > Pet Products > Pet Health Care & Hygiene > Supplements & Vitamins,Beauty & Health > Health Care > Personal Health Care,nan",
    "Jewelry & accessories > necklaces & pendants,Jewelry & Accessories > Customized Jewelry > Customized Necklaces,nan": \
        "Jewelry & Accessories > Necklaces & Pendants,Jewelry & Accessories > Customized Jewelry > Customized Necklaces,nan",
    "Beauty & Health > Skin Care Tools > Jade Roller,Beauty & health > skin care tools > facial care tool,Beauty & Health > Skin Care Tools > Face Lift Devices": \
        "Beauty & Health > Skin Care Tools > Jade Roller,Beauty & Health > Skin Care Tools > Facial Care Tool,Beauty & Health > Skin Care Tools > Face Lift Devices", 
    "Women's Clothing > Tops > Tees,Women's Clothing > Tops > Blouses & Shirts ,nan": \
        "Women's Clothing > Tops > Tees,Women's Clothing > Tops > Blouses & Shirts,nan",
    "Beauty & Health > Skin Care > Face > Cleansers,Beauty & Health > Makeup > Makeup Remover,Beauty & Health > Skin Care > Face > Clean": \
        "Beauty & Health > Skin Care > Face > Cleansers,Beauty & Health > Makeup > Makeup Remover,nan"
}
df_offshore['query_classifications'] = df_offshore.query_classifications.apply(lambda x: x.strip().replace('\n', '').replace('| ',',').replace('|',',')).apply(
    lambda x: manual_correction[x] if x in manual_correction else x)

In [61]:
df_offshore['query_classification_lists'] = df_offshore.query_classifications.apply(trie.extract_from_text)

ERROR:  Apparel Accessories > Girls' Accessories > Girls' Hats,Apparel Accessories > Women's Hats,Apparel Accessories > Girls' Accessories > Girls' Hats, Scarves & Gloves Sets []
ERROR:  Apparel Accessories > Boys' Accessories > Boys' Hats,Apparel Accessories > Girls' Accessories > Girls' Hats,nan []


In [62]:
errors

["Apparel Accessories > Girls' Accessories > Girls' Hats,Apparel Accessories > Women's Hats,Apparel Accessories > Girls' Accessories > Girls' Hats, Scarves & Gloves Sets",
 "Apparel Accessories > Boys' Accessories > Boys' Hats,Apparel Accessories > Girls' Accessories > Girls' Hats,nan"]

In [63]:
recs = []
for i in df_offshore.to_dict('records'):
    if i['query_classifications'] == "Apparel Accessories > Girls' Accessories > Girls' Hats,Apparel Accessories > Women's Hats,Apparel Accessories > Girls' Accessories > Girls' Hats, Scarves & Gloves Sets":
        i['query_classification_lists'] = [
            "Apparel Accessories > Girls' Accessories > Girls' Hats",
            "Apparel Accessories > Women's Hats",
            "Apparel Accessories > Girls' Accessories > Girls' Hats, Scarves & Gloves Sets"
        ]
    if i['query_classifications'] == "Apparel Accessories > Boys' Accessories > Boys' Hats,Apparel Accessories > Girls' Accessories > Girls' Hats,nan":
        i['query_classification_lists'] = [
            "Apparel Accessories > Boys' Accessories > Boys' Hats",
            "Apparel Accessories > Girls' Accessories > Girls' Hats",
            "nan"
        ]
    recs.append(i)
df_offshore = pd.DataFrame(recs)

In [64]:
path2id = {}
for i in df_tax.to_dict('records'):
    path2id[i['category_path'].lower().strip()] = i['id']
    path2id[i['category_path'].strip()] = i['id']

In [65]:
id2path = {}
for i in df_tax.to_dict('records'):
    id2path[i['id']] = i['category_path']

In [66]:
path2id['nan'] = -1
path2id['No Categories Match'] = -1
path2id['no_match'] = -1

In [67]:
df_offshore['query_classification_ids'] = df_offshore['query_classification_lists'].apply(lambda x: [path2id[i] for i in x])

In [68]:
df_appen['query_classification_lists'] = df_appen['Final Answer'].apply(lambda x: x.split('\n'))
df_appen['query_classification_ids'] = df_appen['query_classification_lists'].apply(lambda x: [path2id[i] for i in x])

In [70]:
df_join = df_offshore[['query', 'query_classification_ids']].rename(columns={'query_classification_ids': 'offshore_query_classification_ids'}).merge( 
    df_appen[['query', 'query_classification_ids']].rename(columns={'query_classification_ids': 'appen_query_classification_ids'}), 
    on='query', how='inner'
)

In [71]:
len(df_appen), len(df_offshore), len(df_join)

(13309, 17207, 1593)

In [72]:
def convert_id_to_path(li):
    if set(li) == set([-1]):
        return []
    else:
        res = [] 
        for i in li:
            if int(i) in id2path:
                res.append(id2path[int(i)])
        return res

In [73]:
df_join['offshore_query_classification_lists'] = df_join['offshore_query_classification_ids'].apply(convert_id_to_path).apply(tuple)
df_join['appen_query_classification_lists'] = df_join['appen_query_classification_ids'].apply(convert_id_to_path).apply(tuple)

In [74]:
df_join[['query', 'offshore_query_classification_lists', 'appen_query_classification_lists']].to_excel(
    'analysis/appen_label_quality_check_20230104.xlsx'
)

In [75]:
len(df_join)

1593

In [76]:
from sklearn.metrics import accuracy_score, classification_report

In [77]:
lab_offshore = np.zeros((len(df_join), max(df_tax['id'])))
lab_appen = np.zeros((len(df_join), max(df_tax['id'])))

In [78]:
accuracy_score(
    y_true=df_join['offshore_query_classification_lists'].apply(lambda x: '|'.join(x)),
    y_pred=df_join['appen_query_classification_lists'].apply(lambda x: '|'.join(x))
)

0.28750784682988073

In [79]:
for ind, i in enumerate(df_join.to_dict('records')):
    for j in i['offshore_query_classification_ids']:
        if j != -1:
            lab_offshore[ind][j] = 1. 
    for j in i['appen_query_classification_ids']:
        if j != -1:
            lab_appen[ind][j] = 1. 
    

In [84]:
df_metrics = pd.DataFrame(classification_report(y_true=lab_offshore, y_pred=lab_appen, zero_division=0, output_dict=True)).T

In [85]:
df_metrics

,precision,recall,f1-score,support
0,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,2.0
...,...,...,...,...
6144,0.000000,0.000000,0.000000,3.0
micro avg,0.430917,0.367847,0.396892,2569.0
macro avg,0.080104,0.075046,0.073853,2569.0
weighted avg,0.457353,0.367847,0.383624,2569.0


In [86]:
df_join.offshore_query_classification_ids.apply(lambda x: len([i for i in x if i != -1])).value_counts()

1    865
3    392
2    264
0     72
Name: offshore_query_classification_ids, dtype: int64

In [87]:
df_join.appen_query_classification_ids.apply(lambda x: len([i for i in x if i != -1])).value_counts()

1    1013
2     401
3     126
0      53
Name: appen_query_classification_ids, dtype: int64

# more lenient metrics

In [95]:
df_all_metrics = []
for constraint in range(1,6):
    tmp = df_join.copy()
    tmp['offshore_query_classification_lists'] = tmp['offshore_query_classification_lists'].apply(
        lambda x: tuple([' > '.join(i.split(' > ')[:constraint]) for i in x]))
    tmp['appen_query_classification_lists'] = tmp['appen_query_classification_lists'].apply(
        lambda x: tuple([' > '.join(i.split(' > ')[:constraint]) for i in x]))
    tmp['offshore_query_classification_ids'] = tmp['offshore_query_classification_lists'].apply(lambda x: [path2id[i] for i in x])
    tmp['appen_query_classification_ids'] = tmp['appen_query_classification_lists'].apply(lambda x: [path2id[i] for i in x])
    
    lab_offshore = np.zeros((len(tmp), max(df_tax['id'])))
    lab_appen = np.zeros((len(tmp), max(df_tax['id'])))
    for ind, i in enumerate(tmp.to_dict('records')):
        for j in i['offshore_query_classification_ids']:
            if j != -1:
                lab_offshore[ind][j] = 1. 
        for j in i['appen_query_classification_ids']:
            if j != -1:
                lab_appen[ind][j] = 1. 
    
    df_metrics_tmp = pd.DataFrame(classification_report(y_true=lab_offshore, y_pred=lab_appen, zero_division=0, output_dict=True)).T
    df_metrics_tmp['depth_constraint'] = constraint
    df_all_metrics.append(df_metrics_tmp)
df_all_metrics = pd.concat(df_all_metrics)

In [98]:
df_all_metrics[df_all_metrics.index == 'weighted avg']

,precision,recall,f1-score,support,depth_constraint
weighted avg,0.741434,0.738530,0.733982,1809.0,1
weighted avg,0.648180,0.586340,0.596110,2079.0,2
weighted avg,0.513966,0.428571,0.443682,2436.0,3
weighted avg,0.457920,0.368277,0.384045,2566.0,4
weighted avg,0.457353,0.367847,0.383624,2569.0,5
